# 키워드 검색 - Lexical Search


## 사전 준비


### 패키지 및 데이터 준비


필요한 패키지를 설치합니다.


In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install pandas

워크샵에서 사용할 데이터 파일을 읽어 판다스 데이터프레임으로 저장합니다.


In [1]:
import pandas as pd
import requests

df = pd.read_csv("./data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈


데이터의 스키마와 레코드 수를 확인합니다.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1000 non-null   object 
 1   genre       1000 non-null   object 
 2   year        1000 non-null   int64  
 3   date        1000 non-null   float64
 4   rating      1000 non-null   float64
 5   vote_count  1000 non-null   int64  
 6   plot        1000 non-null   object 
 7   main_act    1000 non-null   object 
 8   supp_act    1000 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


### OpenSearch 도메인에 연결하기


먼저 utils 디렉토리에 있는 공용모듈을 사용하기 위한 준비를 합니다.


In [3]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

이 코드를 통해 OpenSearch 도메인에 연결하는 데 필요한 자격 증명과 엔드포인트 주소를 가져올 수 있습니다. 이 정보를 사용하여 OpenSearch 클라이언트를 초기화하고 인덱싱, 검색 등의 작업을 수행할 수 있습니다.


In [5]:
import boto3

session = boto3.Session()
session.region_name

'us-west-2'

In [ ]:
import boto3, json

# region_name = "us-west-2"
session = boto3.Session()
region_name = session.region_name

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

위의 코드로부터 가져온 호스트 URL과 인증정보를 통해 OpenSearch 도메인에 연결합니다.


In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

연결이 잘 되었는지 확인하기 위해 간단한 요청을 보내보겠습니다. analyze 엔드포인트를 사용하여 nori 분석기로 간단한 한국어를 분석해 봅니다.


In [ ]:
request_body = {"analyzer": "nori", "text": "OpenSearch 워크샵에 오신 고객 여러분 환영합니다."}

# Send the request to the _analyze endpoint
response = aos_client.indices.analyze(body=request_body)

# Print the response
print(json.dumps(response, indent=4, ensure_ascii=False))

### 인덱스 생성


이제 인덱스를 생성합니다. 인덱스의 이름은 movies_lexical 로 하고 맵핑 정보는 다음과 같이 지정합니다.|


In [ ]:
index_name = "movie_lexical"

movie_lexical = {
    "settings": {
        "number_of_replicas": 0,
        "number_of_shards": 1,
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
        }
    },
}

위의 맵핑 정보로 인덱스를 생성합니다.


In [ ]:
aos_client.indices.create(index=index_name, body=movie_lexical)
aos_client.indices.get(index=index_name)

### 데이터 인제스트


생성된 인덱스에 데이터를 인제스트합니다. 여기서는 opensearchpy 패키지에서 제공하는 parallel_bulk를 사용하여 빠르게 데이터를 인제스트합니다.


In [ ]:
from opensearchpy import helpers

# Pandas DataFrame을 JSON 형식의 문자열로 변환
json_data = df.to_json(orient="records", lines=True)
# JSON 문자열을 개별 JSON 객체로 분할하고, 마지막 빈 줄을 제거
docs = json_data.split("\n")[:-1]  # To remove the last empty line


# JSON 객체를 OpenSearch에 업로드할 수 있는 형식으로 변환
def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
# 병렬로 벌크 업로드를 수행
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=20, thread_count=4, queue_size=4
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

데이터가 바로 반영되도록 인덱스를 Refresh하고 인제스트가 잘 되었는지 확인하기 위해 도큐먼트의 총 합을 count로 확인합니다.


In [ ]:
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

이제 키워드 검색을 수행할 모든 준비가 끝났습니다.


## 키워드 검색 결과 확인하기


키워드 검색을 위한 헬퍼 함수를 생성합니다. 여기서는 상위 10개의 결과만을 확인합니다.


In [ ]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "plot", "genre", "main_act"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=["_score", "title", "plot", "genre", "rating", "main_act"],
    )
    display(query_result_df)

자연어로 검색결과를 확인해봅니다.


In [ ]:
keyword_search("지구의 영웅들이 힘을 합쳐 우주의 악당을 물리친다")